# Fetching the journeys data for analysis

Download the CSV data from TfL, and save in a GeoPandas geodataframe.


In [2]:
# Converts journeys into geometry

import os
import csv
import json
import urllib.request
from requests import get

import pandas as pd
import geopandas as gpd
import rasterio
import rasterio.plot
import matplotlib.pyplot as plt

from shapely.geometry import LineString

In [3]:
# set paths
DL_path = "data/cycles/DL_data"
points_path = "data/cycles/points"
points_fn = "BikePoints.geojson"
journeys_path = "data/cycles/journeys"

# set source URL and filename
source_url = "https://cycling.data.tfl.gov.uk/usage-stats/"
source_fn = [
    "346JourneyDataExtract28Nov2022-04Dec2022.csv",
    "347JourneyDataExtract05Dec2022-11Dec2022.csv",
    "348JourneyDataExtract12Dec2022-18Dec2022.csv",
    "349JourneyDataExtract19Dec2022-25Dec2022.csv",
    "350JourneyDataExtract26Dec2022-01Jan2023.csv",
    "351JourneyDataExtract02Jan2023-08Jan2023.csv",
    "352JourneyDataExtract09Jan2023-15Jan2023.csv",
    "353JourneyDataExtract16Jan2023-22Jan2023.csv",
    "354JourneyDataExtract23Jan2023-29Jan2023.csv",
    "355JourneyDataExtract30Jan2023-05Feb2023.csv",
    "356JourneyDataExtract06Feb2023-12Feb2023.csv",
    "357JourneyDataExtract13Feb2023-19Feb2023.csv",
    "358JourneyDataExtract20Feb2023-26Feb2023.csv",
    "359JourneyDataExtract27Feb2023-05Mar2023.csv",
    "360JourneyDataExtract06Mar2023-12Mar2023.csv",
    "361JourneyDataExtract13Mar2023-19Mar2023.csv",
    "362JourneyDataExtract20Mar2023-26Mar2023.csv",
    "363JourneyDataExtract27Mar2023-02Apr2023.csv",
    "364JourneyDataExtract03Apr2023-09Apr2023.csv",
    "365JourneyDataExtract10Apr2023-16Apr2023.csv",
    "366JourneyDataExtract17Apr2023-23Apr2023.csv",
    "367JourneyDataExtract24Apr2023-30Apr2023.csv",
    "368JourneyDataExtract01May2023-07May2023.csv",
    "369JourneyDataExtract08May2023-14May2023.csv",
    "370JourneyDataExtract15May2023-21May2023.csv",
    "371JourneyDataExtract22May2023-28May2023.csv",
    "372JourneyDataExtract29May2023-04Jun2023.csv",
    "373JourneyDataExtract05Jun2023-11Jun2023.csv",
    "374JourneyDataExtract12Jun2023-18Jun2023.csv",
    "375JourneyDataExtract19Jun2023-30Jun2023.csv",
    "376JourneyDataExtract01Jul2023-14Jul2023.csv",
    "377JourneyDataExtract15Jul2023-31Jul2023.csv",
    "378JourneyDataExtract01Aug2023-14Aug2023.csv",
    "378JourneyDataExtract15Aug2023-31Aug2023.csv",
    "379JourneyDataExtract01Sep2023-14Sep2023.csv",
    "380JourneyDataExtract15Sep2023-30Sep2023.csv",
    "381JourneyDataExtract01Oct2023-14Oct2023.csv",
    "382JourneyDataExtract15Oct2023-31Oct2023.csv",
    "383JourneyDataExtract01Nov2023-14Nov2023.csv",
    "384JourneyDataExtract15Nov2023-30Nov2023.csv"
]

In [3]:
# download journeys

# creates saving directory if does not exist
if not os.path.exists(journeys_path):
    os.makedirs(journeys_path)


for fn in source_fn:
    # check if local exists
    if not os.path.exists(os.path.join(journeys_path, fn)):
        source = source_url + fn
        print(f"Downloading   : {fn}")
        with open(os.path.join(journeys_path, fn), "wb") as file:
            response = get(source)
            file.write(response.content)
    else:
        print(f"Found in local: {fn}")

print('Done')

Found in local: 346JourneyDataExtract28Nov2022-04Dec2022.csv
Found in local: 347JourneyDataExtract05Dec2022-11Dec2022.csv
Found in local: 348JourneyDataExtract12Dec2022-18Dec2022.csv
Found in local: 349JourneyDataExtract19Dec2022-25Dec2022.csv
Found in local: 350JourneyDataExtract26Dec2022-01Jan2023.csv
Found in local: 351JourneyDataExtract02Jan2023-08Jan2023.csv
Found in local: 352JourneyDataExtract09Jan2023-15Jan2023.csv
Found in local: 353JourneyDataExtract16Jan2023-22Jan2023.csv
Found in local: 354JourneyDataExtract23Jan2023-29Jan2023.csv
Found in local: 355JourneyDataExtract30Jan2023-05Feb2023.csv
Found in local: 356JourneyDataExtract06Feb2023-12Feb2023.csv
Found in local: 357JourneyDataExtract13Feb2023-19Feb2023.csv
Found in local: 358JourneyDataExtract20Feb2023-26Feb2023.csv
Found in local: 359JourneyDataExtract27Feb2023-05Mar2023.csv
Found in local: 360JourneyDataExtract06Mar2023-12Mar2023.csv
Found in local: 361JourneyDataExtract13Mar2023-19Mar2023.csv
Found in local: 362Journ

## Create dataframe from journeys

Using the above dataset, a dataframe of journeys are created in this script below.


In [17]:
# test flag, set 1 for testing
test_flag = 0


journeys_df = []
# load data
# keeping it as a list to avoid memory crashes in further analysis
# merging will come at the very end
for idx, fn in enumerate(source_fn):
    journeys_df.append(pd.read_csv(os.path.join(journeys_path, fn), low_memory = False))
    if test_flag == 1:
        break


## Join with height data

In [4]:
# set path
points_path = 'data/cycles/points'
points_parquet_fn = 'docking_stations.geoparquet'
journeys_df_fn = 'journeys.parquet'

# load points gdf
points_gdf = gpd.read_parquet(os.path.join(points_path, points_parquet_fn))


In [20]:
# merge the points data to the journeys data

# add prefix to data points
start_points = points_gdf.add_prefix('start_')
end_points = points_gdf.add_prefix('end_')

# create new list
journeys_df_merged = []

for df in journeys_df:     
    # merge the start point data
    temp_df = df.merge(start_points, left_on = 'Start station', right_on = 'start_name')
    # merge the end point data
    temp_df = temp_df.merge(end_points, left_on = 'End station', right_on = 'end_name')

    journeys_df_merged.append(temp_df)

# delete the unnneccesary df
del(start_points)
del(end_points)

journeys_df_merged[0]

,Number,Start date,Start station number,Start station,End date,End station number,End station,Bike number,Bike model,Total duration,...,end_id,end_name,end_ports,end_location,end_zone,end_height,end_LSOA11CD,end_LSOA11NM,end_cc_zone,end_geometry
0,127293434,2022-12-04 23:59,1194,"Westminster University, Marylebone",2022-12-05 00:08,300079,"London Street, Paddington",24140,CLASSIC,9m 5s,...,300079,"London Street, Paddington",19,London Street,Paddington,26.3120,E01033594,Westminster 015G,False,POINT (526821.250 181157.755)
1,127256557,2022-12-02 20:06,1085,"Old Quebec Street, Marylebone",2022-12-02 20:11,300079,"London Street, Paddington",51668,CLASSIC,5m 17s,...,300079,"London Street, Paddington",19,London Street,Paddington,26.3120,E01033594,Westminster 015G,False,POINT (526821.250 181157.755)
2,127222561,2022-12-01 15:48,1085,"Old Quebec Street, Marylebone",2022-12-01 15:54,300079,"London Street, Paddington",57546,CLASSIC,5m 24s,...,300079,"London Street, Paddington",19,London Street,Paddington,26.3120,E01033594,Westminster 015G,False,POINT (526821.250 181157.755)
3,127198699,2022-11-30 18:05,1085,"Old Quebec Street, Marylebone",2022-11-30 18:11,300079,"London Street, Paddington",52833,CLASSIC,6m 37s,...,300079,"London Street, Paddington",19,London Street,Paddington,26.3120,E01033594,Westminster 015G,False,POINT (526821.250 181157.755)
4,127193452,2022-11-30 15:44,1085,"Old Quebec Street, Marylebone",2022-11-30 15:49,300079,"London Street, Paddington",55552,CLASSIC,5m 18s,...,300079,"London Street, Paddington",19,London Street,Paddington,26.3120,E01033594,Westminster 015G,False,POINT (526821.250 181157.755)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159513,127132580,2022-11-28 10:09,300226,"Copper Box Arena, Queen Elizabeth Olympic Park",2022-11-28 10:12,300219,"Here East South, Queen Elizabeth Olympic Park",40028,CLASSIC,2m 52s,...,300219,"Here East South, Queen Elizabeth Olympic Park",28,Here East South,Queen Elizabeth Olympic Park,9.4975,E01001849,Hackney 018D,False,POINT (537350.344 184927.413)
159514,127281432,2022-12-04 10:19,300219,"Here East South, Queen Elizabeth Olympic Park",2022-12-04 10:38,300219,"Here East South, Queen Elizabeth Olympic Park",50956,CLASSIC,18m 42s,...,300219,"Here East South, Queen Elizabeth Olympic Park",28,Here East South,Queen Elizabeth Olympic Park,9.4975,E01001849,Hackney 018D,False,POINT (537350.344 184927.413)
159515,127272102,2022-12-03 16:21,300219,"Here East South, Queen Elizabeth Olympic Park",2022-12-03 17:35,300219,"Here East South, Queen Elizabeth Olympic Park",58934,CLASSIC,1h 14m 9s,...,300219,"Here East South, Queen Elizabeth Olympic Park",28,Here East South,Queen Elizabeth Olympic Park,9.4975,E01001849,Hackney 018D,False,POINT (537350.344 184927.413)
159516,127265715,2022-12-03 12:22,300219,"Here East South, Queen Elizabeth Olympic Park",2022-12-03 13:06,300219,"Here East South, Queen Elizabeth Olympic Park",58934,CLASSIC,44m 3s,...,300219,"Here East South, Queen Elizabeth Olympic Park",28,Here East South,Queen Elizabeth Olympic Park,9.4975,E01001849,Hackney 018D,False,POINT (537350.344 184927.413)


In [ ]:
%time

# get the geometry for each journey
for i, df in enumerate(journeys_df_merged):
    print(f'Working on df:{i}')
    # define the geometry column
    df['geometry'] = df.apply(lambda row: LineString([row['start_geometry'], row['end_geometry']]), axis=1)
    
    # define the line geometry
#    for index, row in df.iterrows():
    #    if row['start_name'] != row['end_name']:
    #        df.loc[index, 'geometry'] = LineString([row['start_geometry'], row['end_geometry']])
    #    else:
    #        df.loc[index, 'geometry'] = row['start_geometry']        

CPU times: user 8 µs, sys: 0 ns, total: 8 µs
Wall time: 15 µs
Working on df:0
Working on df:1
Working on df:2
Working on df:3
Working on df:4
Working on df:5
Working on df:6
Working on df:7
Working on df:8
Working on df:9
Working on df:10
Working on df:11
Working on df:12
Working on df:13
Working on df:14
Working on df:15
Working on df:16
Working on df:17
Working on df:18
Working on df:19
Working on df:20
Working on df:21
Working on df:22
Working on df:23
Working on df:24
Working on df:25
Working on df:26
Working on df:27
Working on df:28
Working on df:29
Working on df:30
Working on df:31
Working on df:32
Working on df:33
Working on df:34
Working on df:35


In [ ]:
# make one large df from the list of df
df_concat = pd.DataFrame()

for df in journeys_df_merged:
    pd.concat([df_concat, df])

In [ ]:
# create geodataframe
columns = ['Number', 'Start date', 'Start station number', 'start_location', 'start_zone', 
           'start_LSOA11CD', 'start_LSOA11NM', 'start_cc_zone', 'start_height',
           'End date', 'End station number', 'end_location', 'end_zone', 
           'end_LSOA11CD', 'end_LSOA11NM', 'end_cc_zone', 'end_height', 
           'Bike number', 'Bike model', 'Total duration (ms)', 'height_diff', 'geometry']

journeys_gdf = gpd.GeoDataFrame(df_concat[columns], crs = 'EPSG:27700')

In [ ]:
# clean data

# clean dates
dates = ['Start date', 'End date']
for d in dates:
    journeys_gdf[d] = pd.to_datetime(journeys_gdf[d], format = '%Y-%m-%d %H:%M')

# make bike model into categorical data
# this should only have 2 types: 'CLASSIC' and 'PBSC_EBIKE'
journeys_gdf['Bike model'] = journeys_gdf['Bike model'].astype('category')

In [ ]:
# calculate the height difference
journeys_gdf['height_diff'] = journeys_gdf.end_height - journeys_gdf.start_height

# save the distance of journey as a column
journeys_gdf['distance'] = journeys_gdf.length

In [ ]:
journeys_gdf.height_diff.describe()

In [ ]:
journeys_gdf_fn = 'journeys_gdf.geoparquet'

journeys_gdf.to_parquet(os.path.join(journeys_path, journeys_gdf_fn))